In [1]:
import pandas as pd
import numpy as np

In [2]:
from datasets import load_dataset

ds = load_dataset("lucadiliello/newsqa")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/681 [00:00<?, ?B/s]

data/train-00000-of-00001-ec54fbe500fc3b(…):   0%|          | 0.00/29.7M [00:00<?, ?B/s]

data/validation-00000-of-00001-3cf888b12(…):   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/74160 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4212 [00:00<?, ? examples/s]

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers', 'key', 'labels'],
        num_rows: 74160
    })
    validation: Dataset({
        features: ['context', 'question', 'answers', 'key', 'labels'],
        num_rows: 4212
    })
})

In [4]:
ds['train'][1]['labels']

[{'end': [269], 'start': [261]}]

In [5]:
#questions = [q.strip() for q in ds['train']["question"]]


In [6]:
def prepare_train_features_fixed(examples, max_length=384, doc_stride=128, pad_on_right=True):
    # Clean question text
    examples["question"] = [q.strip() for q in examples["question"]]

    # Tokenize with stride for long contexts
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",  # ✅ critical
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_idx = sample_mapping[i]

        # use your 'labels' column
        answer = examples["labels"][sample_idx][0]
        start_char = answer["start"][0]
        end_char = answer["end"][0]

        # determine where context tokens start/end
        context_start = next(i for i, s in enumerate(sequence_ids) if s == (1 if pad_on_right else 0))
        context_end = len(sequence_ids) - 1 - next(i for i, s in enumerate(reversed(sequence_ids)) if s == (1 if pad_on_right else 0))

        # if answer not in this tokenized span → map to CLS
        if not (offsets[context_start][0] <= start_char and offsets[context_end][1] >= end_char):
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue

        # find token start index
        token_start_index = context_start
        while token_start_index <= context_end and offsets[token_start_index][0] <= start_char:
            token_start_index += 1
        token_start_index -= 1

        # find token end index
        token_end_index = context_end
        while token_end_index >= context_start and offsets[token_end_index][1] >= end_char:
            token_end_index -= 1
        token_end_index += 1

        start_positions.append(token_start_index)
        end_positions.append(token_end_index)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    return tokenized_examples



In [7]:
ds["train"]["labels"]

Column([[{'end': [295], 'start': [294]}], [{'end': [269], 'start': [261]}], [{'end': [638], 'start': [624]}], [{'end': [216], 'start': [195]}], [{'end': [216], 'start': [195]}]])

In [8]:
from transformers import RobertaTokenizerFast

model_name = "roberta-base"
tokenizer = RobertaTokenizerFast.from_pretrained(model_name)
tokenizer.is_fast

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

True

In [9]:
tokenized_ds = ds.map(
    prepare_train_features_fixed,
    batched=True,
    remove_columns=ds["train"].column_names
)


Map:   0%|          | 0/74160 [00:00<?, ? examples/s]

Map:   0%|          | 0/4212 [00:00<?, ? examples/s]

In [10]:

i = 25

example = ds["train"][i]


tokenized_example = tokenized_ds["train"][i]

input_ids = tokenized_example["input_ids"]
start = tokenized_example["start_positions"]
end = tokenized_example["end_positions"]

decoded_answer = tokenizer.decode(input_ids[start:end+1]).strip()

start_char = example["labels"][0]["start"][0]
end_char = example["labels"][0]["end"][0]
true_answer = example["context"][start_char:end_char+1].strip()


decoded_context = tokenizer.decode(input_ids)

print(" Question:", example["question"])
print(" True answer (from context):", true_answer)
print(" Decoded predicted answer span:", decoded_answer)
print("—" * 80)
print("Context snippet:", decoded_context[:500])
print("Match ✅" if decoded_answer == true_answer else "Mismatch ❌")


 Question: What were the families ordered to do?
 True answer (from context): Islam or face death.
 Decoded predicted answer span: Islam or face death
————————————————————————————————————————————————————————————————————————————————
Context snippet: <s>What did extremists order thdm to do</s></s>BAGHDAD, Iraq (CNN)  -- At least 6,000 Christians have fled the northern Iraqi city of Mosul in the past week because of killings and death threats, Iraq's Ministry of Immigration and Displaced Persons said Thursday.



A Christian family that fled Mosul found refuge in the Al-Sayida monastery about 30 miles north of the city.



The number represents 1,424 families, at least 70 more families than were reported to be displaced on Wednesday.



The m
Mismatch ❌


In [11]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 208890
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 11800
    })
})

In [12]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained('roberta-base')

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from transformers import TrainingArguments

args = TrainingArguments(
    "roberta-base-finetuned",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    fp16=True,
    gradient_accumulation_steps=2,



)



In [17]:
from transformers import Trainer
from transformers import DefaultDataCollator

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,

)

/tmp/ipython-input-1530036348.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
trainer.train()

Step,Training Loss
500,1.362000
1000,1.064400
1500,0.993000
2000,0.947900
2500,0.919500
3000,0.892300
3500,0.910200
4000,0.887400
4500,0.842500
5000,0.840100


TrainOutput(global_step=13056, training_loss=0.8389148957589093, metrics={'train_runtime': 4225.6379, 'train_samples_per_second': 49.434, 'train_steps_per_second': 3.09, 'total_flos': 4.093670963593728e+16, 'train_loss': 0.8389148957589093, 'epoch': 1.0})

In [19]:
tokenized_ds["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 208890
})

In [20]:
trainer.save_model("/content/roberta-qa-finetuned")


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model = AutoModelForQuestionAnswering.from_pretrained("/content/roberta-qa-finetuned")
tokenizer = AutoTokenizer.from_pretrained("/content/roberta-qa-finetuned")


In [66]:


qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

context = "amey have music club audition tommorow and he's gona get fucked real bad"
question = ["what does amey have tommorow?"]

result = qa_pipeline(
    question=question,
    context=context
)
print(result["answer"])


Device set to use cuda:0


music club audition
